In [1]:
import numpy as np
import pandas as pd
from neo4j import GraphDatabase
from typing import List, Dict
import os
import streamlit as st
import re
import pandas as pd
import ast
from ast import literal_eval

## Main DataBase Class

In [10]:
class CourseDatabase:
    def __init__(self, uri: str, username: str, password: str):
        """Initialize connection to Neo4j database."""
        self.driver = GraphDatabase.driver(uri, auth=(username, password))

    def close(self):
        """Close the database connection."""
        self.driver.close()

    # David's Functions
    def add_course(self, index, title="", units=0, description=""):
        """
        Adds new course node given input or updates corresponding course node given input. 
        """
        with self.driver.session() as session:
            result = session.run("""
                MERGE (c:Course {course_id: $index})
                SET c.name = $index,
                    c.title = CASE 
                        WHEN $title <> '' AND c.title = ''
                        THEN $title
                        ELSE COALESCE(c.title, $title)
                    END,
                    c.units = CASE 
                        WHEN $units <> 0 AND c.units = 0
                        THEN $units
                        ELSE COALESCE(c.units, $units)
                    END,
                    c.description = CASE 
                        WHEN $description <> '' AND c.description = ''
                        THEN $description 
                        ELSE COALESCE(c.description, $description)
                    END
                RETURN c
                """,
                index=index,
                title=title,
                units=units,
                description=description
            )
            return result.single()
    
    def add_milestone(self, milestone_id, title="", units_required=0, description=""):
        with self.driver.session() as session:
            result = session.run("""
                MERGE (m:Milestone {milestone_id: $milestone_id})
                SET m.name = $milestone_id,
                    m.title = CASE
                        WHEN $title <> '' AND m.title = ''
                        THEN $title
                        ELSE COALESCE(m.title, $title)
                    END,
                    m.units_required = CASE 
                        WHEN $units <> 0 AND m.units = 0
                        THEN $units
                        ELSE COALESCE(m.units, $units)
                    END,
                    m.description = CASE 
                        WHEN $description <> '' AND m.description = ''
                        THEN $description 
                        ELSE COALESCE(m.description, $description)
                    END
                RETURN m
                """,
                milestone_id=milestone_id,
                title=title,
                units=units_required,
                description=description
            )
            return result.single()

    def add_or_group(self, group_id):
        with self.driver.session() as session:
            result = session.run("""
                MERGE (og:OrGroup {group_id: $group_id})
                SET og.name = $group_id
                RETURN og
                """,
                group_id=group_id
            )
            return result.single()
    
    def add_milestone_to_milestone(self, milestone_id, minor_milestone_id):
        with self.driver.session() as session:
            try:
                # Check if relationship already exists before creating
                result = session.run("""
                    MATCH (m:Milestone {milestone_id: $milestone_id})
                    MATCH (mm:Milestone {milestone_id: $minor_milestone_id})
                    WITH m, mm
                    WHERE NOT EXISTS((mm)-[:REQUIRED]->(m))
                    MERGE (mm)-[r:REQUIRED]->(m)
                    RETURN r
                    """,
                    milestone_id=milestone_id, minor_milestone_id=minor_milestone_id
                )
                return result.single()
            except Exception as e:
                print(f"Error creating required relation from {minor_milestone_id} to {milestone_id}: {str(e)}")

    def add_group_to_course(self, course_id, group_id):
        with self.driver.session() as session:
            try:
                # Check if relationship already exists before creating
                result = session.run("""
                    MATCH (c:Course {course_id: $course_id})
                    MATCH (og:OrGroup {group_id: $group_id})
                    WITH c, og
                    WHERE NOT EXISTS((og)-[:REQUIRED]->(c))
                    MERGE (og)-[r:REQUIRED]->(c)
                    RETURN r
                    """,
                    course_id=course_id, group_id=group_id
                )
                return result.single()
            except Exception as e:
                print(f"Error creating prerequisite relation from {group_id} to {course_id}: {str(e)}")

    def add_course_to_group(self, group_id, course_id):
        with self.driver.session() as session:
            try:
                # Check if relationship already exists before creating
                result = session.run("""
                    MATCH (c:Course {course_id: $course_id})
                    MATCH (og:OrGroup {group_id: $group_id})
                    WITH c, og
                    WHERE NOT EXISTS((c)-[:REQUIRED]->(og))
                    MERGE (c)-[r:REQUIRED]->(og)
                    RETURN r
                    """,
                    course_id=course_id, group_id=group_id
                )
                return result.single()
            except Exception as e:
                print(f"Error creating required relation from {course_id} to {group_id}: {str(e)}")

    def add_course_in_group(self, course_id, group_id):
        with self.driver.session() as session:
            try:
                # Check if relationship already exists before creating
                result = session.run("""
                    MATCH (c:Course {course_id: $course_id})
                    MATCH (og:OrGroup {group_id: $group_id})
                    WITH c, og
                    WHERE NOT EXISTS((c)-[:INCLUDED_IN]->(og))
                    MERGE (c)-[r:INCLUDED_IN]->(og)
                    RETURN r
                    """,
                    course_id=course_id, group_id=group_id
                )
                return result.single()
            except Exception as e:
                print(f"Error creating course to group relation from {course_id} to {group_id}: {str(e)}")
    
    def add_group_in_milestone(self, group_id, milestone_id):
        with self.driver.session() as session:
            try:
                # Check if relationship already exists before creating
                result = session.run("""
                    MATCH (m:Milestone {milestone_id: $milestone_id})
                    MATCH (og:OrGroup {group_id: $group_id})
                    WITH m, og
                    WHERE NOT EXISTS((og)-[:INCLUDED_IN]->(m))
                    MERGE (og)-[r:INCLUDED_IN]->(m)
                    RETURN r
                    """,
                    group_id=group_id, milestone_id=milestone_id
                )
                return result.single()
            except Exception as e:
                print(f"Error creating included in relation from {group_id} to {milestone_id}: {str(e)}")

    def add_course_in_milestone(self, course_id, milestone_id):
        with self.driver.session() as session:
            try:
                # Check if relationship already exists before creating
                result = session.run("""
                    MATCH (m:Milestone {milestone_id: $milestone_id})
                    MATCH (c:Course {course_id: $course_id})
                    WITH m, c
                    WHERE NOT EXISTS((c)-[:INCLUDED_IN]->(m))
                    MERGE (c)-[r:INCLUDED_IN]->(m)
                    RETURN r
                    """,
                    course_id=course_id, milestone_id=milestone_id
                )
                return result.single()
            except Exception as e:
                print(f"Error creating included in relation from {course_id} to {milestone_id}: {str(e)}")
    
    def get_course_info(self, course_id):
        with self.driver.session() as session:
            result = session.run("""
                MATCH (c:Course {course_id: $course_id})
                RETURN 
                    c.course_id as id,
                    c.title as title,
                    c.units as units,
                    c.description as description
            """, course_id=course_id)
            
            record = result.single()
            if record:
                return {
                    'course_id': record['id'],
                    'title': record['title'],
                    'units': record['units'],
                    'description': record['description']
                }
            return None

    def get_prerequisites(self, course_id):
        with self.driver.session() as session:
            # Query all prerequisites groups and their courses
            result = session.run("""
                MATCH (c:Course {course_id: $course_id})
                MATCH (og:OrGroup)-[:REQUIRED]->(c)
                MATCH (prereq:Course)-[:INCLUDED_IN]->(og)
                RETURN og.group_id as group_id, 
                    collect(prereq.course_id) as prereq_courses
                ORDER BY og.group_id
            """, course_id=course_id)
            
            # Convert result to list of lists format
            prereqs = []
            for record in result:
                prereqs.append(record['prereq_courses'])
                
            return prereqs
    
    

## Testing the get_prereqs function

In [11]:
uri = st.secrets["NEO4J_URI"]
username = st.secrets["NEO4J_USERNAME"]
password = st.secrets["NEO4J_PASSWORD"]

db = CourseDatabase(
    uri=uri,
    username=username,
    password=password
)

## Creating Categories for MATH CS major

In [12]:
# David's Functions
def add_major_requirements(major_req_data):
    # add the meta milestones
    for idx, row in major_req_data.iloc[:3,:].iterrows():
        db.add_milestone(
            row['Milestones'],
            row['Title'],
            row['Units Required'],
            row['Descriptions']
        )

        milestone_requirements = literal_eval(row['Requirements'])
        for group_idx, or_group in enumerate(milestone_requirements):
            print(or_group)
            db.add_milestone(or_group)
            db.add_milestone_to_milestone(row['Milestones'], or_group)
        print(f"added {row['Milestones']} to database.")

    # add the individual milestones and courses
    for idx, row in major_req_data.iloc[3:,:].iterrows():
        db.add_milestone(
            row['Milestones'],
            row['Title'],
            row['Units Required'],
            row['Descriptions']
        )

        milestone_requirements = literal_eval(row['Requirements'])
        for group_idx, or_group in enumerate(milestone_requirements):
            if isinstance(or_group, tuple):
                group_id = f"{row['Milestones']}_PATH_{group_idx}"
                db.add_or_group(group_id)
                db.add_group_in_milestone(group_id, row['Milestones'])
                print(f'{or_group}: added new group')

                for required_course in or_group:
                    db.add_course(required_course)
                    db.add_course_to_group(group_id, required_course)
            elif isinstance(or_group, str):
                db.add_course(or_group)
                db.add_course_in_milestone(or_group, row['Milestones'])
                print(f'{or_group}: directly added course')
        print(f"added {row['Milestones']} to database.")
        
    print("Successfully added all requirement milestones.")


In [13]:
# David's Function
def add_department_catalog(dpt_cat_data):
    """
    Loads entire course catalog into neo4j db. 
    dpt_cat_data should include columns: ['Course_Index', 'Course_Title', 'Course_Units', 'Course_Description', 'Course_Prerequisites']
    """
    for idx, row in dpt_cat_data.iterrows():
        db.add_course(
            row['Course_Index'],
            row['Course_Title'],
            row['Course_Units'],
            row['Course_Description']
        )

        course_prereq_lst = literal_eval(row['Course_Prerequisites'])
        for group_idx, or_group in enumerate(course_prereq_lst):
            # Create unique group ID combining course and group number
            group_id = f"{row['Course_Index']}_GROUP_{group_idx}"
            print(group_id)
            # Add OrGroup
            db.add_or_group(group_id)

            # Add prerequisite relation from OrGroup to main course
            db.add_group_to_course(row['Course_Index'], group_id)

            # Add all prerequisite courses and their relations to OrGroup
            for prereq_course in or_group:
                print(prereq_course)
                # Add prerequisite course (will be ignored if already exists)
                db.add_course(prereq_course)  # Minimal info for prereq courses

                # Add relation from prerequisite course to OrGroup
                db.add_course_in_group(prereq_course, group_id)
        
        print(f"added {row['Course_Index']} to database.")
    print('\nSucessfully added all courses to database.\n')

In [14]:
math_data = pd.read_csv('data/processed/Math Course Data.csv')
add_department_catalog(math_data)

MATH 2_GROUP_0
added MATH 2 to database.
MATH 3B_GROUP_0
added MATH 3B to database.
MATH 3C_GROUP_0
MATH 3B
added MATH 3C to database.
MATH 4C_GROUP_0
MATH 3C
added MATH 4C to database.
MATH 10A_GROUP_0
MATH 3C
MATH 10A_GROUP_1
MATH 4C
added MATH 10A to database.
MATH 10B_GROUP_0
MATH 10A
MATH 10B_GROUP_1
MATH 20A
added MATH 10B to database.
MATH 10C_GROUP_0
MATH 10B
MATH 10C_GROUP_1
MATH 20B
added MATH 10C to database.
MATH 11_GROUP_0
MATH 10B
MATH 11_GROUP_1
MATH 20B
added MATH 11 to database.
MATH 15A_GROUP_0
CSE 8B
MATH 15A_GROUP_1
CSE 11
added MATH 15A to database.
MATH 18_GROUP_0
MATH 4C
MATH 18_GROUP_1
MATH 10A
MATH 18_GROUP_2
MATH 20A
added MATH 18 to database.
MATH 20A_GROUP_0
MATH 2C
MATH 20A_GROUP_1
MATH 4C
MATH 20A_GROUP_2
MATH 10A
added MATH 20A to database.
MATH 20B_GROUP_0
MATH 20A
MATH 20B_GROUP_1
MATH 10B
MATH 20B_GROUP_2
MATH 10C
added MATH 20B to database.
MATH 20C_GROUP_0
MATH 20B
added MATH 20C to database.
MATH 20D_GROUP_0
MATH 20C
MATH 20D_GROUP_1
MATH 21C
MATH 2

In [15]:
cse_data = pd.read_csv('data/processed/CSE Course Data.csv')
add_department_catalog(cse_data)

CSE 3_GROUP_0
added CSE 3 to database.
CSE 4GS_GROUP_0
MATH 10A
MATH 20A
added CSE 4GS to database.
CSE 6GS_GROUP_0
MATH 10A
MATH 20A
added CSE 6GS to database.
CSE 6R_GROUP_0
added CSE 6R to database.
CSE 8A_GROUP_0
added CSE 8A to database.
CSE 8B_GROUP_0
CSE 8A
added CSE 8B to database.
CSE 11_GROUP_0
added CSE 11 to database.
CSE 12_GROUP_0
CSE 8B
CSE 11
added CSE 12 to database.
CSE 15L_GROUP_0
CSE 8B
CSE 11
CSE 12
DSC 30
added CSE 15L to database.
CSE 20_GROUP_0
CSE 11
CSE 6R
CSE 8A
CSE 8B
ECE 15
added CSE 20 to database.
CSE 21_GROUP_0
CSE 20
MATH 15A
MATH 31CH
added CSE 21 to database.
CSE 29_GROUP_0
CSE 11
CSE 8B
ECE 15
added CSE 29 to database.
CSE 30_GROUP_0
CSE 15L
CSE 29
ECE 15
added CSE 30 to database.
CSE 42_GROUP_0
added CSE 42 to database.
CSE 86_GROUP_0
CSE 12
added CSE 86 to database.
CSE 87_GROUP_0
added CSE 87 to database.
CSE 89_GROUP_0
added CSE 89 to database.
CSE 90_GROUP_0
added CSE 90 to database.
CSE 91_GROUP_0
added CSE 91 to database.
CSE 95_GROUP_0
added 

In [17]:
# David's Function
mathcs_req = pd.read_csv('data/processed/MA30 requirements.csv')
add_major_requirements(mathcs_req)

MA30 Lower Division
MA30 Upper Division
added MA30 to database.
MA30_Lower_1
MA30_Lower_2
MA30_Lower_3
MA30_Lower_4
added MA30 Lower Division to database.
MA30_Upper_1
MA30_Upper_2
MA30_Upper_3
MA30_Upper_4
MA30_Upper_5
MA30_Upper_6
MA30_Upper_7
MA30_Upper_8
MA30_Upper_9
MA30_Upper_10
added MA30 Upper Division to database.
('MATH 18', 'MATH 20A', 'MATH 20B', 'MATH 20C', 'MATH 20D', 'MATH 20E'): added new group
('MATH 31AH', 'MATH 31BH', 'MATH 31CH', 'MATH 20D'): added new group
added MA30_Lower_1 to database.
('CSE 8A', 'CSE 8B'): added new group
('CSE 11',): added new group
added MA30_Lower_2 to database.
CSE 15L: directly added course
CSE 29: directly added course
added MA30_Lower_3 to database.
CSE 12: directly added course
added MA30_Lower_4 to database.
MATH 109: directly added course
added MA30_Upper_1 to database.
('MATH 103A', 'MATH 103B'): added new group
('MATH 100A', 'MATH 100B'): added new group
added MA30_Upper_2 to database.
CSE 105: directly added course
added MA30_Upper

In [18]:
db.close()